# Lesson 2. Missing Values
Missing values happen. Be prepared for this common challenge in real datasets.

In [3]:
# So module imports can work and reuse code
import sys; sys.path.insert(0, '../../')

## Tutorial

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv('../../input/melbourne-housing-snapshot/melb_data.csv')

# Select target
y = data.Price

# To keep things simple, we'll use only numerical predictors
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

### Score from Approach 1 (Drop Columns with Missing Values)

In [6]:
from lib.model_score import score_dataset

# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop columns with missing values):
183550.22137772635


### Score from Approach 2 (Imputation)

In [7]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
178166.46269899711


### Score from Approach 3 (An Extension to Imputation)

In [8]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
178927.503183954


**So, why did imputation perform better than dropping the columns?**

The training data has 10864 rows and 12 columns, where three columns contain missing data. For each column, less than half of the entries are missing. Thus, dropping the columns removes a lot of useful information, and so it makes sense that imputation would perform better.

In [9]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


## Exercise

In [10]:
# Read the data
X_full = pd.read_csv('../../input/home-data-for-ml-course/train.csv', index_col='Id')
X_test_full = pd.read_csv('../../input/home-data-for-ml-course/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll use only numerical predictors
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [11]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


### Step 1: Preliminary investigation

In [12]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [25]:
missing_val_columns = missing_val_count_by_column[missing_val_count_by_column > 0]

In [26]:
# Fill in the line below: How many rows are in the training data?
num_rows = X_train.shape[0] 

# Fill in the line below: How many columns in the training data
# have missing values?
num_cols_with_missing = missing_val_columns.size

# Fill in the line below: How many missing entries are contained in 
# all of the training data?
tot_missing = missing_val_columns.sum()


Since there are relatively few missing entries in the data (the column with the greatest percentage of missing values is missing less than 20% of its entries), we can expect that dropping columns is unlikely to yield good results. This is because we'd be throwing away a lot of valuable data, and so imputation will likely perform better.

### Step 2: Drop columns with missing values

In [37]:
# Fill in the line below: get names of columns with missing values
columns_with_missing = missing_val_count_by_column[missing_val_count_by_column > 0].index.to_list() # Your code here

# Fill in the lines below: drop columns in training and validation data
reduced_X_train = X_train.drop(labels= columns_with_missing, axis=1)
reduced_X_valid = X_valid.drop(labels= columns_with_missing, axis=1)

In [36]:
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
18866.728767123288


**JUAAAAAYYYY**???? 18866.728767123288 <------

### Step 3: Imputation

In [40]:
# Fill in the lines below: imputation
my_imputer = SimpleImputer() # Your code here
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Fill in the lines below: imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [41]:
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputation):
19255.558333333334


**JUAAAAAYYYY**???? 19255.558333333334 <------

Given that thre are so few missing values in the dataset, we'd expect imputation to perform better than dropping columns entirely. However, we see that dropping columns performs slightly better! While this can probably partially be attributed to noise in the dataset, another potential explanation is that the imputation method is not a great match to this dataset. That is, maybe instead of filling in the mean value, it makes more sense to set every missing value to a value of 0, to fill in the most frequently encountered value, or to use some other method. For instance, consider the GarageYrBlt column (which indicates the year that the garage was built). It's likely that in some cases, a missing value could indicate a house that does not have a garage. Does it make more sense to fill in the median value along each column in this case? Or could we get better results by filling in the minimum value along each column? It's not quite clear what's best in this case, but perhaps we can rule out some options immediately - for instance, setting missing values in this column to 0 is likely to yield horrible results!

### Step 4: Generate test predictions 

#### Part A
Use the next code cell to preprocess the training and validation data. Set the preprocessed DataFrames to final_X_train and final_X_valid. You can use any approach of your choosing here! in order for this step to be marked as correct, you need only ensure:

- the preprocessed DataFrames have the same number of columns,
- the preprocessed DataFrames have no missing values,
- final_X_train and y_train have the same number of rows, and
- final_X_valid and y_valid have the same number of rows

In [46]:
missing_val_columns

LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64

In [45]:
X_train.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
MSSubClass,1168.0,56.605308,42.172322,20.0,20.00,50.0,70.00,190.0
LotFrontage,956.0,69.614017,22.946069,21.0,59.00,69.0,80.00,313.0
LotArea,1168.0,10589.672945,10704.180793,1300.0,7589.50,9512.5,11601.50,215245.0
OverallQual,1168.0,6.086473,1.367472,1.0,5.00,6.0,7.00,10.0
OverallCond,1168.0,5.572774,1.116908,1.0,5.00,5.0,6.00,9.0
YearBuilt,1168.0,1970.890411,30.407486,1872.0,1953.75,1972.0,2000.00,2010.0
YearRemodAdd,1168.0,1984.692637,20.684612,1950.0,1966.00,1993.0,2004.00,2010.0
MasVnrArea,1162.0,103.481067,182.676225,0.0,0.00,0.0,167.75,1600.0
BsmtFinSF1,1168.0,439.890411,435.106803,0.0,0.00,379.5,716.00,2260.0
BsmtFinSF2,1168.0,45.571918,156.229962,0.0,0.00,0.0,0.00,1120.0


From the description txt file, the columns wih missing values are:

- LotFrontage    212, LotFrontage: Linear feet of street connected to property
- MasVnrArea       6, MasVnrArea: Masonry veneer area in square feet
- GarageYrBlt     58, GarageYrBlt: Year garage was built

-> LotFrontage could be described with LotArea, since the amount of data is small we could remove this column,

-> For MasVnrArea, only 6 records are missing, here impute values can be more usefull, since the data we are replacing is small compared with the one we will miss if we drop the column.

-> GarageYrBlt, could be related to GarageArea. It could be that this column is set to NaN for properties that does not have any garage. Let us explore this case

In [47]:
X_train.loc[X_train['GarageYrBlt'].isna(), ['GarageArea','GarageYrBlt']]

,GarageArea,GarageYrBlt
Id,,
432,0,NaN
639,0,NaN
1338,0,NaN
529,0,NaN
961,0,NaN
1012,0,NaN
1284,0,NaN
1039,0,NaN
166,0,NaN


In [48]:
X_train.loc[X_train['GarageArea'] == 0, ['GarageArea','GarageYrBlt']]

,GarageArea,GarageYrBlt
Id,,
432,0,NaN
639,0,NaN
1338,0,NaN
529,0,NaN
961,0,NaN
1012,0,NaN
1284,0,NaN
1039,0,NaN
166,0,NaN


Seems this is the case, so we will proceed to add the lowest value to the `GarageYrBlt`

In [54]:
# Drop `LotFrontage` column
drop_X_train = X_train.drop(labels=['LotFrontage'], axis=1)
drop_X_valid = X_valid.drop(labels=['LotFrontage'], axis=1)

# Set to `GarageYrBlt` = 0 for missing values
drop_X_train.loc[drop_X_train['GarageYrBlt'].isna(), ['GarageYrBlt']] = 0
drop_X_valid.loc[drop_X_valid['GarageYrBlt'].isna(), ['GarageYrBlt']] = 0

# Impute `MasVnrArea` column
imputer_MasVnrArea = SimpleImputer() # Your code here
final_X_train = pd.DataFrame(imputer_MasVnrArea.fit_transform(drop_X_train))
final_X_valid = pd.DataFrame(imputer_MasVnrArea.transform(drop_X_valid))

# Fill in the lines below: imputation removed column names; put them back
final_X_train.columns = drop_X_train.columns
final_X_valid.columns = drop_X_valid.columns


,count,mean,std,min,25%,50%,75%,max
MSSubClass,1168.0,56.605308,42.172322,20.0,20.00,50.0,70.00,190.0
LotArea,1168.0,10589.672945,10704.180793,1300.0,7589.50,9512.5,11601.50,215245.0
OverallQual,1168.0,6.086473,1.367472,1.0,5.00,6.0,7.00,10.0
OverallCond,1168.0,5.572774,1.116908,1.0,5.00,5.0,6.00,9.0
YearBuilt,1168.0,1970.890411,30.407486,1872.0,1953.75,1972.0,2000.00,2010.0
YearRemodAdd,1168.0,1984.692637,20.684612,1950.0,1966.00,1993.0,2004.00,2010.0
MasVnrArea,1168.0,103.481067,182.206015,0.0,0.00,0.0,166.25,1600.0
BsmtFinSF1,1168.0,439.890411,435.106803,0.0,0.00,379.5,716.00,2260.0
BsmtFinSF2,1168.0,45.571918,156.229962,0.0,0.00,0.0,0.00,1120.0
BsmtUnfSF,1168.0,568.049658,437.569919,0.0,228.00,482.5,811.25,2153.0


In [56]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
# Define and fit model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

# Get validation predictions and MAE
preds_valid = model.predict(final_X_valid)
print("MAE (Your approach):")
print(mean_absolute_error(y_valid, preds_valid))

MAE (Your approach):
17988.90197488584


**JUAAAAAAAAAAAAY** 🥲

In [57]:
# Fill in the line below: preprocess test data
# Drop `LotFrontage` column
drop_X_test = X_test.drop(labels=['LotFrontage'], axis=1)

# Set to `GarageYrBlt` = 0 for missing values
drop_X_test.loc[drop_X_test['GarageYrBlt'].isna(), ['GarageYrBlt']] = 0

# Impute `MasVnrArea` column
imputer_MasVnrArea = SimpleImputer() # Your code here
final_X_test = pd.DataFrame(imputer_MasVnrArea.fit_transform(drop_X_test))

# Fill in the lines below: imputation removed column names; put them back
final_X_test.columns = drop_X_test.columns

# Fill in the line below: get test predictions
preds_test = model.predict(final_X_test)

In [ ]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)